# `FC.py` |  cofound regressions

In [1]:
# Load libraries
import sys
import os
import glob
import numpy as np
import nibabel as nib
from sklearn.linear_model import LinearRegression
import warnings

# Load data from single subject
funcDir='/data_/mica3/BIDS_MICs/derivatives/micapipe/sub-PX079/ses-01/func/desc-se_task-rest_acq-AP_bold'
func_lab='_space-func_desc-se'
# Load confound files
os.chdir(funcDir+'/volumetric/')
x_spike = " ".join(glob.glob(funcDir+'/volumetric/'+'*spikeRegressors_FD.1D'))
x_dof = " ".join(glob.glob(funcDir+'/volumetric/*'+func_lab+'.1D'))
# x_refmse = " ".join(glob.glob(funcDir+'/volumetric/'+'*metric_REFMSE.1D'))
x_fd = " ".join(glob.glob(funcDir+'/volumetric/'+'*metric_FD*'))
x_csf = " ".join(glob.glob(funcDir+'/volumetric/'+'*CSF*'))
x_wm = " ".join(glob.glob(funcDir+'/volumetric/'+'*WM*'))
x_gs = " ".join(glob.glob(funcDir+'/volumetric/'+'*global*'))

# define functions
def expand_dim(Data):
    if Data.ndim == 1:
        Data = np.expand_dims(Data, axis=1)
    return Data

# Set paths to files
dof = np.loadtxt(x_dof)
csf = expand_dim(np.loadtxt(x_csf))
wm = expand_dim(np.loadtxt(x_wm))
gs = expand_dim(np.loadtxt(x_gs))
spike = expand_dim(np.loadtxt(x_spike))

ones = np.ones((wm.shape[0], 1))
mdl = np.append(np.append(np.append(np.append(ones, dof, axis=1), wm, axis=1), csf, axis=1), gs, axis = 1)

# load cortica data
os.chdir(funcDir+'/surf/')
x_lh = " ".join(glob.glob(funcDir+'/surfaces/'+'*space-conte69-32k_lh_10mm*'))
x_rh = " ".join(glob.glob(funcDir+'/surfaces/'+'*space-conte69-32k_rh_10mm*'))
lh_data = nib.load(x_lh)
lh_data = np.squeeze(lh_data.get_fdata())
rh_data = nib.load(x_rh)
rh_data = np.squeeze(rh_data.get_fdata())

# Reformat data
data = []
data = np.transpose(np.append(lh_data, rh_data, axis=0))
n_vertex_ctx_c69 = data.shape[1]
del lh_data
del rh_data

OSError:  not found.

In [ ]:
# mod.fit(x, y), where y=[time series: time,vertex] and x=[regressor matrix: time, regresor] 
# Y ~ X (new stuff)
slmYX = LinearRegression().fit(mdl, data)
# residuals NO intercept
res_YX = data - np.dot(mdl, slmYX.coef_.T) # <<<<----------- Here
# residuals with intercept
res_YXI =  data - (np.dot(mdl, slmYX.coef_.T) + slmYX.intercept_.T)
# residuals with function predict(X)
residual = (data - slmYX.predict(mdl))

# residuals function is EQUAL to residuals with intercept
np.array_equal(res_YXI, residual)
# AND same results as using R function: residuals(lm(y~x))

# residuals functio is NOT equal to dot product of coefficients*X
np.array_equal(res_YX, residual)

# Are the methods the same?? almost
res_corr = np.zeros(695)
for i in range(0,695):
    res_corr[i] = np.corrcoef(res_YX[i,:], residual[i,:])[0,1]
np.mean(res_corr)

# --------------------------------------------------------------------
# X ~ Y 
slmXY = LinearRegression().fit(data, mdl)
# residuals NO intercep
res_XY = data - np.dot(mdl, slmXY.coef_)

# residuals with function predict(X) is not possible bc
# the intercep is 10.
# residual_XY = data - (np.dot(mdl, slmXY.coef_) + slmXY.intercept_)


# Are the methods the same?? almost
res_diff = np.zeros(695)
for i in range(0,695):
    res_diff[i] = np.corrcoef(residual[i,:], res_XY[i,:])[0,1]
np.mean(res_diff)